In this notebook, I will perform data analysis of Kaggle Challenge KKBox Churn classifier. 

KKBox is one of the biggest music streaming company in chinese world. The raw data contains about 60Gb and 10Millium
user activites. Due to the large data volume, it will be tricky to analyze with native pandas (chunk batch operation must be used to avoid memory drain.) Therefore I will use Dask, a pandas-like parallel computing analysis package to handle the job. In the end, I will perform ensemble method to build a classify model to predict user churn.

Table of content.
1. First look of data - variable explore
2. Feature engineering
3. First level classifier
4. Second level classifier

In [62]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import dask.dataframe as dd

The raw data is three files, (and transactions_v2, user_logs_v2 for data in March)
members: gender, city, registered_via, registration_init_time, expiration_date (member histroy)
transaction: including payment method_id, payment_plan_days, plan_list_price, actual_amount_paid, is_auto_renew
user_logs: daily user logs, this will be the most challenge and interesting info about the user, including number of song played 25%, 50%, 75% of song lenght, #unique song play, total listening sec.
Each file is connect by unique user id 'msno'. The data including user behavier from 2015-1-1 to 2017-2-28, which is huge amount of information we can dig into. I will try to analyze it in the second part see if we can find more features for prediction. 

The train and test set data are saved in 'train_v2', where we will use data in Feb to predict churn user in March.
(Update: now the competition had been change the target to churn users in Aprail)
The train and test set are seperated by transaction date. In this first part I will set down modeling framework.
Let's first find out the trainset and testset.

Definition of train and testset

In [63]:
#df_1 = dd.read_csv('train.csv').set_index('msno')
df_2 = dd.read_csv('train_v2.csv').set_index('msno')
df_member = dd.read_csv('members_v3.csv',parse_dates = ['registration_init_time']).set_index('msno') #all data
#df_trans1 = dd.read_csv('transactions.csv',parse_dates = ['transaction_date','membership_expire_date']).set_index('msno')
df_trans2 = dd.read_csv('transactions_v2.csv',parse_dates = ['transaction_date','membership_expire_date']).set_index('msno')
#df_log1 = dd.read_csv('user_logs.csv',parse_dates=['date']).set_index('msno')
df_log2 = dd.read_csv('user_logs_v2.csv',parse_dates=['date']).set_index('msno')

In [74]:
# df_pd1 = pd.read_csv('train.csv')
# df_pd2 = pd.read_csv('train_v2.csv')
# d1_msno = df_pd1.msno
# overlap = df_pd2[df_pd2.msno.isin(d1_msno)]
sub = pd.read_csv('sample_submission_v2.csv')
sub.info()
# sub = pd.read_csv('sample_submission_zero.csv')
# sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907471 entries, 0 to 907470
Data columns (total 2 columns):
msno        907471 non-null object
is_churn    907471 non-null int64
dtypes: int64(1), object(1)
memory usage: 13.8+ MB


Because of the change of competition, we have churn info on Feb and March now. Now I have two way to go, take Feb data predict March and directly see the logloss score. Or I can take March data to predict Aprail. Lets do second. and try to figure out how to combine two dataset for more precise model.

In [64]:
df_total = dd.merge(df_2,df_member, left_index=True,right_index=True,how='left')
# df_total = dd.merge(df_total,df_trans2, left_index=True,how='left')
# df_total = dd.merge(df_total,df_log2,left_index=True,how='left')

In [65]:
describe = df_total.describe().compute()

/Users/yhhsu/Anaconda/anaconda/envs/ph2t/lib/python3.6/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/Users/yhhsu/Anaconda/anaconda/envs/ph2t/lib/python3.6/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [66]:
describe

,is_churn,city,bd,registered_via
count,970960.000000,860967.000000,860967.000000,860967.000000
mean,0.089942,5.937295,13.583854,6.892744
std,0.286099,6.441203,20.064040,1.937794
min,0.000000,1.000000,-3152.000000,3.000000
25%,0.000000,NaN,NaN,NaN
50%,0.000000,NaN,NaN,NaN
75%,0.000000,4.000000,26.000000,3.000000
max,1.000000,22.000000,2016.000000,13.000000


In [68]:
df_total.head(npartitions=5)

,is_churn,city,bd,gender,registered_via,registration_init_time
msno,,,,,,
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,1.0,0.0,NaN,7.0,2014-06-08
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,15.0,26.0,male,9.0,2010-11-18
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0,1.0,0.0,NaN,7.0,2014-09-27
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0,18.0,21.0,male,7.0,2016-03-15
++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0,22.0,32.0,male,9.0,2012-01-22


In [73]:
df_total.gender.value_counts().compute()

male      204561
female    184344
Name: gender, dtype: int64

In [81]:
df_total[df_total.city.isnull()].head(npartitions=5)

,is_churn,city,bd,gender,registered_via,registration_init_time
msno,,,,,,
++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,0,NaN,NaN,NaN,NaN,NaT
++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,NaN,NaN,NaN,NaN,NaT
++2Ju1OdxLSyexwhZ/C0glNK0DMIfUjsFpk9lt8Dll8=,0,NaN,NaN,NaN,NaN,NaT
++4wuAZmfzMBjRHP4vDpTk+jkj9Xam8SW5rNJrGFEsE=,0,NaN,NaN,NaN,NaN,NaT
++5XBxbJNz3idK9egVoGbWJ4xhGQu2MR0pSP0kL9DK4=,0,NaN,NaN,NaN,NaN,NaT


In [82]:
df_total2 = dd.merge(df_total,df_trans2,right_index=True,left_index=True,how='left')

In [83]:
df2_describe = df_total2.describe().compute()

/Users/yhhsu/Anaconda/anaconda/envs/ph2t/lib/python3.6/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [84]:
df2_describe

,is_churn,city,bd,registered_via,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel
count,1.169418e+06,1.052582e+06,1.052582e+06,1.052582e+06,1.132036e+06,1.132036e+06,1.132036e+06,1.132036e+06,1.132036e+06,1.132036e+06
mean,1.222899e-01,6.256564e+00,1.435392e+01,6.905842e+00,3.887682e+01,3.356030e+01,1.455336e+02,1.450239e+02,9.163012e-01,2.904766e-02
std,3.276205e-01,6.538549e+00,2.034310e+01,1.946620e+00,3.877857e+00,3.179832e+01,1.331882e+02,1.333376e+02,2.769356e-01,1.679402e-01
min,0.000000e+00,1.000000e+00,-3.152000e+03,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.000000e+00,4.000000e+00,2.600000e+01,3.000000e+00,4.000000e+01,3.000000e+01,1.490000e+02,1.490000e+02,1.000000e+00,0.000000e+00
max,1.000000e+00,2.200000e+01,2.016000e+03,1.300000e+01,4.100000e+01,4.500000e+02,2.000000e+03,2.000000e+03,1.000000e+00,1.000000e+00


In [ ]:
df_2 

In [ ]:
label = gender_count.index
count = gender_count.values
plt.figure(figsize=(8,8))
plt.pie(count,labels=label,autopct='%1.1f%%',shadow=True,radius=20)
plt.axis('equal')
plt.show()

In [ ]:
# 326 unique value, 0 as largest group 
# bd : age some outlier shows in the data
mask = df_member.db > 100, 
df_member.bd.loc[df.memory_usage].value_counts().compute()

In [ ]:
df_member.bd.isnull().sum().compute()

In [ ]:
df_member.gender